In [2]:
import os
import inspect
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
from tracr.rasp import rasp
from tracr.compiler import compiling, rasp_to_graph
import numpy as np
from tracr.compiler.validating import validate
from typing import Union, TypeVar
from rasp_generator import map_primitives, sampling, utils
import networkx as nx

from tracr.rasp.rasp import Map, SequenceMap, LinearSequenceMap, Select, Aggregate, Comparison, SelectorWidth, indices, tokens 


def compile_rasp_to_model(sop: rasp.SOp, vocab={1,2,3,4}, max_seq_len=5, compiler_bos="BOS"):
    return compiling.compile_rasp_to_model(
        sop,
        vocab=vocab,
        max_seq_len=max_seq_len,
        compiler_bos=compiler_bos
    )


TEST_INPUT = [1,2,3,4]

# Program Generation

In [3]:
sampler = sampling.ProgramSampler(validate_compilation=True)
errs = sampler.sample()
print(sampler.program([1,2,3,4]))
print()
print("Retries encountered:")
for err in errs:
    print(err)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


[False, False, True, False]

Retries encountered:
Maximum retries reached. Could not sample categorical aggregate with valid output domain.This because the sampler couldn't find a selector with width 1, and other sampled selectors don't result in an output domain that is a subset of the input domain.
No SOps of type float in scope.
No SOps of type float in scope.


In [14]:
# print program with and without test input
k = 1
utils.print_program(sampler.sops[-k])
print()
print()
utils.print_program(sampler.sops[-k], TEST_INPUT)

select_46 = Select(tokens, tokens, predicate=Comparison.EQ)
select_44 = Select(tokens, indices, predicate=Comparison.EQ)
aggregate_45 = Aggregate(select_46, tokens)    # type: categorical
aggregate_43 = Aggregate(select_44, aggregate_45)    # type: categorical
sequence_map_42 = SequenceMap(lambda x, y: x*y, tokens, aggregate_43)    # type: categorical


select_46 = Select(tokens, tokens, predicate=Comparison.EQ)
select_44 = Select(tokens, indices, predicate=Comparison.EQ)
aggregate_45 = Aggregate(select_46, tokens)    # output: [1, 2, 3, 4]
aggregate_43 = Aggregate(select_44, aggregate_45)    # output: [None, 1, 2, 3]
sequence_map_42 = SequenceMap(lambda x, y: x*y, tokens, aggregate_43)    # output: [None, 2, 6, 12]
